# Mortgage Data From the Home Mortgage Disclosure Act
I'm really interested in housing data. The Consumer Financial Production Bureau makes a LOT of that kind of thing available for free, but the datasets are enormous. I'm still figuring out the best way for me to pursue my analysis.<br>
<br>
But in the meantime, I was able to download loan application records for mortgages in 2023 and do a bit of refining for the analysis I want to conduct.<br><br>
According to [its website](https://ffiec.cfpb.gov/data-publication/snapshot-national-loan-level-dataset/2023), The Snapshot National Loan Level Dataset files contain the national HMDA datasets as of May 1, 2024 for all HMDA reporters, as modified by the Bureau to protect applicant and borrower privacy. The snapshot files are available to download in both .csv and pipe delimited text file formats.
<br>
Use caution when analyzing loan amount and income, which do not have an upper limit and may contain outliers.
<br><br>
**Source**<br>
“HMDA - Home Mortgage Disclosure Act.” 2024. Cfpb.gov. 2024. https://ffiec.cfpb.gov/data-publication/snapshot-national-loan-level-dataset/2023.

## 2023
#### Snapshot (Pennsylvania)
I think it's reasonable to do one state at the time. I have some personal interest in Pennsylvania, so I'll start there.

In [3]:
# I probably won't need all of these for this analysis, but this is a comprehensive list of the libraries I ever need

import requests
from datetime import datetime
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import confusion_matrix

In [3]:
# From the source url (listed in Cell 1), I downloaded loan application records for 2023.

# The .csv is huge, and sometimes my system doesn't like reading it all at once.
# A work-around is the attractively named process of chunking.

# First create an output list to store filtered chunks
filtered_chunks = []

# Then define the chunk size. I usually try 10K to start, but you can adjust based on your available memory.
chunk_size = 10_000

# Read the CSV file in chunks, and like my NFIP project, I usually look at these .csvs by state. I like to start with Pennsylvania.
# Here's my for loop to read the .csv in chunks:

for chunk in pd.read_csv('2023_public_lar_csv.csv', chunksize = chunk_size, low_memory = False):
    
    # I don't need to read the entire file, just rows where state_code is 'PA'
    filtered_chunk = chunk[chunk['state_code'] == 'PA']
    
    # And I append each filtered chunk to the output list we defined up top
    filtered_chunks.append(filtered_chunk)

# After the whole thing is read, concatenate all filtered chunks into a single DataFrame
lar23 = pd.concat(filtered_chunks, ignore_index = True)

# Optional: Save filtered data to a new CSV file so if I need to go back and redo some cleaning, I can some of the waiting
lar23.to_csv('lar23.csv', index = False)

# And lastly I'll check the size of this PA-filtered .csv for data-wrangling in Jupyter
lar23.shape

(423016, 99)

In [5]:
# Well, 423K rows isn't so bad.

# Let's see what columns I'm working with
# I know there will be 99...

lar23.columns

Index(['activity_year', 'lei', 'derived_msa_md', 'state_code', 'county_code',
       'census_tract', 'conforming_loan_limit', 'derived_loan_product_type',
       'derived_dwelling_category', 'derived_ethnicity', 'derived_race',
       'derived_sex', 'action_taken', 'purchaser_type', 'preapproval',
       'loan_type', 'loan_purpose', 'lien_status', 'reverse_mortgage',
       'open_end_line_of_credit', 'business_or_commercial_purpose',
       'loan_amount', 'combined_loan_to_value_ratio', 'interest_rate',
       'rate_spread', 'hoepa_status', 'total_loan_costs',
       'total_points_and_fees', 'origination_charges', 'discount_points',
       'lender_credits', 'loan_term', 'prepayment_penalty_term',
       'intro_rate_period', 'negative_amortization', 'interest_only_payment',
       'balloon_payment', 'other_nonamortizing_features', 'property_value',
       'construction_method', 'occupancy_type',
       'manufactured_home_secured_property_type',
       'manufactured_home_land_property_in

In [7]:
# I created a function to focus the data on that's most important to me:
# Approved applications only,
# For primary residence,
# No reverse morgages,
# No business or commercial purposes,
# No interest-only payments,
# Occupancy types no greater than 4 units,
# And Pennsylvania county codes only.

def filter_lar(df):
    filtered_df = df[
        (df['action_taken'] == 1) &
        (df['loan_purpose'] == 1) &
        (df['reverse_mortgage'].isin([2, 1111])) &
        (df['business_or_commercial_purpose'].isin([2, 1111])) &
        (df['interest_only_payment'].isin([2, 1111])) &
        (df['occupancy_type'] == 1) &
        (df['total_units'].isin([1, 2, 3, 4])) &
        (df['county_code'].astype(str).str.startswith('42'))
    ]
    
    # Then I dropped the columns I don't need. (Hopefully I'll ditch more as I go forward, but this helps for now.)
    columns_to_drop = [
        'derived_msa_md', 'state_code', 'census_tract', 'derived_dwelling_category', 
        'conforming_loan_limit', 'action_taken', 'loan_purpose', 'lien_status', 
        'reverse_mortgage', 'hoepa_status', 'negative_amortization', 'interest_only_payment', 
        'occupancy_type', 'submission_of_application', 'initially_payable_to_institution',
        'denial_reason_1', 'denial_reason_2', 'denial_reason_3', 'denial_reason_4'
    ]
    
    filtered_df = filtered_df.drop(columns = columns_to_drop, errors = 'ignore')
    
    return filtered_df

In [9]:
# Now I apply the function to lar23
lar23_filtered = filter_lar(lar23)

# And see what kind of shape I'm looking at now
lar23_filtered.shape

(47759, 80)

In [11]:
# Whoa. That eliminated almost 90% of my rows! I almost want to go back and expand my dataset...
# But no. This is just 2023 PA data, and it's still nearly 50K datapoints. Ideally I'd like to concatenate more years and/or states,
# so I'll stick to these narrowing parameters.

# Let's preview the dataframe
lar23_filtered.head()

,activity_year,lei,county_code,derived_loan_product_type,derived_ethnicity,derived_race,derived_sex,purchaser_type,preapproval,loan_type,...,aus_3,aus_4,aus_5,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
1255,2023,549300RN01LBYR8ZVX74,42125.0,Conventional:First Lien,Not Hispanic or Latino,White,Female,0,2,1,...,NaN,NaN,NaN,5319,7.56,101900,127.88,1698,1929,43
1261,2023,549300RN01LBYR8ZVX74,42027.0,Conventional:First Lien,Not Hispanic or Latino,White,Joint,71,2,1,...,NaN,NaN,NaN,2983,18.44,115500,119.02,967,1393,51
1266,2023,549300RN01LBYR8ZVX74,42043.0,FHA:First Lien,Hispanic or Latino,Race Not Available,Female,71,2,2,...,NaN,NaN,NaN,6484,40.72,102700,106.86,1388,1929,42
1269,2023,549300RN01LBYR8ZVX74,42087.0,Conventional:First Lien,Not Hispanic or Latino,White,Joint,0,2,1,...,NaN,NaN,NaN,2372,8.26,77800,53.55,613,959,73
1273,2023,549300RN01LBYR8ZVX74,42061.0,Conventional:First Lien,Not Hispanic or Latino,White,Female,0,2,1,...,NaN,NaN,NaN,4624,5.95,77800,90.06,1508,2212,51


In [13]:
# Right away I see something I want to fix, but first I gotta shorten this variable name; I'll be using it a lot.

lar = lar23_filtered

In [15]:
# Great. Now it looks like the county_codes are floats. Am I right?

lar.county_code.dtype

dtype('float64')

In [17]:
# Yep. I wish FIPS codes always came to us as strings; I've never had a case where I needed them to be a numeric value.
# So first I convert the numeric value to strings.

lar['county_code'] = lar['county_code'].astype(int).astype(str)

# And check that I did it properly
lar.county_code.dtype

dtype('O')

In [19]:
# One thing done. But did this transfer to the main dataset?

lar.head()

,activity_year,lei,county_code,derived_loan_product_type,derived_ethnicity,derived_race,derived_sex,purchaser_type,preapproval,loan_type,...,aus_3,aus_4,aus_5,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
1255,2023,549300RN01LBYR8ZVX74,42125,Conventional:First Lien,Not Hispanic or Latino,White,Female,0,2,1,...,NaN,NaN,NaN,5319,7.56,101900,127.88,1698,1929,43
1261,2023,549300RN01LBYR8ZVX74,42027,Conventional:First Lien,Not Hispanic or Latino,White,Joint,71,2,1,...,NaN,NaN,NaN,2983,18.44,115500,119.02,967,1393,51
1266,2023,549300RN01LBYR8ZVX74,42043,FHA:First Lien,Hispanic or Latino,Race Not Available,Female,71,2,2,...,NaN,NaN,NaN,6484,40.72,102700,106.86,1388,1929,42
1269,2023,549300RN01LBYR8ZVX74,42087,Conventional:First Lien,Not Hispanic or Latino,White,Joint,0,2,1,...,NaN,NaN,NaN,2372,8.26,77800,53.55,613,959,73
1273,2023,549300RN01LBYR8ZVX74,42061,Conventional:First Lien,Not Hispanic or Latino,White,Female,0,2,1,...,NaN,NaN,NaN,4624,5.95,77800,90.06,1508,2212,51


In [21]:
# GORGEOUS. What an efficient way to turn my FIPS floats into strings.
# For now, I'll save this to a .csv, whether I'm done for now or not. If my computer crashes, I don't want to have to run that humungous 2023 LAR .csv again.
# Let's throw the shape down here for posterity first.

lar.shape

(47759, 80)

In [23]:
# And now preserve this cleaned file as a .csv

lar.to_csv('lar_filtered_pa_23.csv', index = False)

## 2022 is a bigger file.
### Will my chunking `for` loop work to read it?

In [26]:
filtered_chunks = []

chunk_size = 10_000

for chunk in pd.read_csv('2022_public_lar_csv.csv', chunksize = chunk_size, low_memory = False):
    filtered_chunk = chunk[chunk['state_code'] == 'PA']
    filtered_chunks.append(filtered_chunk)

lar22 = pd.concat(filtered_chunks, ignore_index = True)

lar22.to_csv('lar22.csv', index = False)

lar22.shape

(585665, 99)

In [29]:
# Holy smokes, it did.
# Well, I already know what columns it has (and which I don't need).
# How many rows do we think my filter function will eliminate? Will it be 90% again?

lar22_filtered = filter_lar(lar22)

# And see what kind of shape I'm looking at now
lar22_filtered.shape

(62117, 80)

In [31]:
# Nice.

# OK, let's shorten that variable name and repeat the cleaning we did from 2023.
# I'll keep a 22 on this one since I don't want to risk losing my 2023 data in lar.
# In fact, let's rename lar to lar23 to keep things organized and confusion-free. (Or at least confusion-minimal.)

lar23 = lar
lar22 = lar22_filtered

# Now we can take care of that pesky county_code column

lar22['county_code'] = lar22['county_code'].astype(int).astype(str)
lar22.head()

,activity_year,lei,county_code,derived_loan_product_type,derived_ethnicity,derived_race,derived_sex,purchaser_type,preapproval,loan_type,...,aus_3,aus_4,aus_5,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
1,2022,RVDPPPGHCGZ40J4VQ731,42101,Conventional:First Lien,Not Hispanic or Latino,Asian,Female,1,2,1,...,NaN,NaN,NaN,4826,94.76,75000,25.0,507,2330,67
7,2022,RVDPPPGHCGZ40J4VQ731,42045,Conventional:First Lien,Not Hispanic or Latino,White,Joint,3,2,1,...,NaN,NaN,NaN,3417,52.53,75000,133.0,886,1124,75
8,2022,RVDPPPGHCGZ40J4VQ731,42029,Conventional:First Lien,Not Hispanic or Latino,White,Joint,3,2,1,...,NaN,NaN,NaN,4956,12.53,131400,165.0,1852,2000,30
9,2022,RVDPPPGHCGZ40J4VQ731,42133,Conventional:First Lien,Ethnicity Not Available,Race Not Available,Female,3,2,1,...,NaN,NaN,NaN,3295,14.78,93300,97.0,1018,1229,17
20,2022,RVDPPPGHCGZ40J4VQ731,42069,Conventional:First Lien,Not Hispanic or Latino,Asian,Male,1,2,1,...,NaN,NaN,NaN,4163,50.47,74100,47.0,735,1341,68


In [33]:
# Beautiful. Throw the shape down here...

lar22.shape

(62117, 80)

In [35]:
# Good to see it's the same as right after we ran the filter function.
# So my converting the county_codes from floats to strings didn't mess with the shape.

# Let's .csv this baby!

lar22.to_csv('lar_filtered_pa_22.csv', index = False)

# Do I dare try 2021?

## 2021
### I guess I do.
##### One kinda bummer is that HMDA has more detailed data for 2022 and 2021, the one-year files that incorporate adjustments to the 2021 national HMDA datasets, submitted as of May 1, 2023. For 2021, that includes all updates to the Loan Application Register (LAR) and Transmittal Sheet (TS) made in the 12 months following the 2021 reporting deadline of March 1, 2022.
##### But the 2022 Snapshot was still bigger than the 2023 one. And I've had enough heartburn trying to read big files from CFPB before.
##### I'll stick to Snapshot files. (For now.)

In [42]:
# Heads up! CFPB changed the naming convention here. The 2021 Public LAR Snapshot downloads as 2021_public_lar.csv 
# whereas 2022 and 2023 had "_csv" at the end of the file name
# I missed that the first time and got an error code due to the file name not existing in my directory

filtered_chunks = []

chunk_size = 10_000

for chunk in pd.read_csv('2021_public_lar.csv', chunksize = chunk_size, low_memory = False):
    filtered_chunk = chunk[chunk['state_code'] == 'PA']
    filtered_chunks.append(filtered_chunk)

lar21 = pd.concat(filtered_chunks, ignore_index = True)

lar21.to_csv('lar21.csv', index = False)

lar21.shape

(861415, 99)

In [45]:
# 860K, we're getting close to 1 million applications per year. Let's filter that baby.

lar21_filtered = filter_lar(lar21)

lar21_filtered.shape

(82091, 80)

In [47]:
# Nice, down even more than 90%. These will concatenate nicely.

lar21 = lar21_filtered

lar21['county_code'] = lar21['county_code'].astype(int).astype(str)
lar21.head()

,activity_year,lei,county_code,derived_loan_product_type,derived_ethnicity,derived_race,derived_sex,purchaser_type,preapproval,loan_type,...,aus_3,aus_4,aus_5,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
9586,2021,5493008NWHQT1R22C024,42129,FHA:First Lien,Not Hispanic or Latino,White,Female,71,2,2,...,NaN,NaN,NaN,1990,9.20,84200,84.0,598,919,76
9587,2021,5493008NWHQT1R22C024,42101,FHA:First Lien,Not Hispanic or Latino,Black or African American,Female,9,2,2,...,NaN,NaN,NaN,5806,92.83,67500,27.0,629,1945,76
9592,2021,5493008NWHQT1R22C024,42045,Conventional:First Lien,Not Hispanic or Latino,White,Joint,71,2,1,...,NaN,NaN,NaN,6468,13.81,67500,144.0,1594,2226,67
9593,2021,5493008NWHQT1R22C024,42101,FHA:First Lien,Not Hispanic or Latino,Black or African American,Female,71,2,2,...,NaN,NaN,NaN,2912,100.00,67500,58.0,627,1376,76
9597,2021,5493008NWHQT1R22C024,42029,Conventional:First Lien,Not Hispanic or Latino,White,Female,71,2,1,...,NaN,NaN,NaN,3635,12.27,116300,136.0,1085,1385,32


In [49]:
lar21.to_csv('lar_filtered_pa_21.csv', index = False)

## 2020
##### For 2020 I actually have to use the One-Year dataset, as Snapshot isn't available.
##### Fingers crossed it reads!

In [52]:
filtered_chunks = []

chunk_size = 10_000

for chunk in pd.read_csv('2020_public_lar_one_year.csv', chunksize = chunk_size, low_memory = False):
    filtered_chunk = chunk[chunk['state_code'] == 'PA']
    filtered_chunks.append(filtered_chunk)

lar20 = pd.concat(filtered_chunks, ignore_index = True)

lar20.to_csv('lar20.csv', index = False)

lar20.shape

(792007, 99)

In [54]:
# Well we're going down in size.

lar20_filtered = filter_lar(lar20)

lar20_filtered.shape

(67237, 80)

In [56]:
lar20 = lar20_filtered

# It occurs to me I could have waited till I concatenated all the years and only had to convert the county_codes to strings once.
# I imagine I'll be doing more formatting/cleaning with the eventual concatenated file.
# But when I started it with 2023, I didn't know if I'd be able to use more years.
# Oh well! It's not too many more keystrokes. And I think I have to stop at 2018; that's when the data storage format changes.
# It's totally feasible to fetch pre-2018 data, and with a little attention you can merge it with 2018 and after,
# but that takes a little more work than I want to do on data cleaning/wrangling for this current free-time experimenting.
# I'd rather dive into some visualizations and analysis. If my findings prompt me to check earlier data, I will.

lar20['county_code'] = lar20['county_code'].astype(int).astype(str)

In [58]:
lar20.to_csv('lar_filtered_pa_20.csv', index = False)

## 2019

In [61]:
filtered_chunks = []

chunk_size = 10_000

for chunk in pd.read_csv('2019_public_lar_one_year.csv', chunksize = chunk_size, low_memory = False):
    filtered_chunk = chunk[chunk['state_code'] == 'PA']
    filtered_chunks.append(filtered_chunk)

lar19 = pd.concat(filtered_chunks, ignore_index = True)

lar19.to_csv('lar19.csv', index = False)

lar19.shape

(598793, 99)

In [64]:
lar19_filtered = filter_lar(lar19)

lar19_filtered.shape

(55224, 80)

In [66]:
lar19 = lar19_filtered

lar19['county_code'] = lar19['county_code'].astype(int).astype(str)

lar19.to_csv('lar_filtered_pa_19.csv', index = False)

## 2018
##### I'm actually moving into the Three-Year Dataset now. The Three-Year files incorporate adjustments to the 2018 national HMDA datasets, submitted as of December 31, 2021. They include all updates to the Loan Application Register (LAR) and Transmittal Sheet (TS) made in the 36 months following the 2018 reporting deadline of March 1, 2019. Files are available to download in both .csv and pipe delimited text file formats.
##### I'll have to check if this one includes a different array of columns than the previous files.

In [69]:
# Note that this file was saved with an extra underscore. As my downloading, it arrived with this file name: 2018_public__lar_three_year.csv

filtered_chunks = []

chunk_size = 10_000

for chunk in pd.read_csv('2018_public__lar_three_year.csv', chunksize = chunk_size, low_memory = False):
    filtered_chunk = chunk[chunk['state_code'] == 'PA']
    filtered_chunks.append(filtered_chunk)

lar18 = pd.concat(filtered_chunks, ignore_index = True)

lar18.to_csv('lar18.csv', index = False)

lar18.shape

(548976, 99)

In [71]:
# Well we have 99 columns. Let's look at what they're called.

lar18.columns

Index(['activity_year', 'lei', 'derived_msa_md', 'state_code', 'county_code',
       'census_tract', 'conforming_loan_limit', 'derived_loan_product_type',
       'derived_dwelling_category', 'derived_ethnicity', 'derived_race',
       'derived_sex', 'action_taken', 'purchaser_type', 'preapproval',
       'loan_type', 'loan_purpose', 'lien_status', 'reverse_mortgage',
       'open_end_line_of_credit', 'business_or_commercial_purpose',
       'loan_amount', 'combined_loan_to_value_ratio', 'interest_rate',
       'rate_spread', 'hoepa_status', 'total_loan_costs',
       'total_points_and_fees', 'origination_charges', 'discount_points',
       'lender_credits', 'loan_term', 'prepayment_penalty_term',
       'intro_rate_period', 'negative_amortization', 'interest_only_payment',
       'balloon_payment', 'other_nonamortizing_features', 'property_value',
       'construction_method', 'occupancy_type',
       'manufactured_home_secured_property_type',
       'manufactured_home_land_property_in

In [73]:
# To refresh my memory, here are the columns the 2023 Snapshot read in with:

# Index(['activity_year', 'lei', 'derived_msa_md', 'state_code', 'county_code',
#       'census_tract', 'conforming_loan_limit', 'derived_loan_product_type',
#       'derived_dwelling_category', 'derived_ethnicity', 'derived_race',
#       'derived_sex', 'action_taken', 'purchaser_type', 'preapproval',
#       'loan_type', 'loan_purpose', 'lien_status', 'reverse_mortgage',
#       'open_end_line_of_credit', 'business_or_commercial_purpose',
#       'loan_amount', 'combined_loan_to_value_ratio', 'interest_rate',
#       'rate_spread', 'hoepa_status', 'total_loan_costs',
#       'total_points_and_fees', 'origination_charges', 'discount_points',
#       'lender_credits', 'loan_term', 'prepayment_penalty_term',
#       'intro_rate_period', 'negative_amortization', 'interest_only_payment',
#       'balloon_payment', 'other_nonamortizing_features', 'property_value',
#       'construction_method', 'occupancy_type',
#       'manufactured_home_secured_property_type',
#       'manufactured_home_land_property_interest', 'total_units',
#       'multifamily_affordable_units', 'income', 'debt_to_income_ratio',
#       'applicant_credit_score_type', 'co_applicant_credit_score_type',
#       'applicant_ethnicity_1', 'applicant_ethnicity_2',
#       'applicant_ethnicity_3', 'applicant_ethnicity_4',
#       'applicant_ethnicity_5', 'co_applicant_ethnicity_1',
#       'co_applicant_ethnicity_2', 'co_applicant_ethnicity_3',
#       'co_applicant_ethnicity_4', 'co_applicant_ethnicity_5',
#       'applicant_ethnicity_observed', 'co_applicant_ethnicity_observed',
#       'applicant_race_1', 'applicant_race_2', 'applicant_race_3',
#       'applicant_race_4', 'applicant_race_5', 'co_applicant_race_1',
#       'co_applicant_race_2', 'co_applicant_race_3', 'co_applicant_race_4',
#       'co_applicant_race_5', 'applicant_race_observed',
#       'co_applicant_race_observed', 'applicant_sex', 'co_applicant_sex',
#       'applicant_sex_observed', 'co_applicant_sex_observed', 'applicant_age',
#       'co_applicant_age', 'applicant_age_above_62',
#       'co_applicant_age_above_62', 'submission_of_application',
#       'initially_payable_to_institution', 'aus_1', 'aus_2', 'aus_3', 'aus_4',
#       'aus_5', 'denial_reason_1', 'denial_reason_2', 'denial_reason_3',
#       'denial_reason_4', 'tract_population',
#       'tract_minority_population_percent',
#       'ffiec_msa_md_median_family_income', 'tract_to_msa_income_percentage',
#       'tract_owner_occupied_units', 'tract_one_to_four_family_homes',
#       'tract_median_age_of_housing_units'],
#      dtype='object')

In [75]:
# I think we're good to run the filter function

lar18_filtered = filter_lar(lar18)

lar18_filtered.shape

(52963, 80)

In [77]:
lar18 = lar18_filtered

lar18['county_code'] = lar18['county_code'].astype(int).astype(str)

lar18.to_csv('lar_filtered_pa_18.csv', index = False)

# Me Oh My
# I'm ready to concatenate.

In [7]:
lar23 = pd.read_csv('lar_filtered_pa_23.csv', low_memory = False)

In [9]:
lar22 = pd.read_csv('lar_filtered_pa_22.csv', low_memory = False)

In [11]:
lar21 = pd.read_csv('lar_filtered_pa_21.csv', low_memory = False)

In [13]:
lar20 = pd.read_csv('lar_filtered_pa_20.csv', low_memory = False)

In [15]:
lar19 = pd.read_csv('lar_filtered_pa_19.csv', low_memory = False)

In [17]:
lar18 = pd.read_csv('lar_filtered_pa_18.csv', low_memory = False)

In [19]:
lar1823 = pd.concat([lar23, lar22, lar21, lar20, lar19, lar18], ignore_index = True)

In [21]:
lar1823.shape

(367391, 80)

In [23]:
lar1823.to_csv('lar_filtered_pa_18-23.csv', index = False)

In [25]:
lar1823.head()

,activity_year,lei,county_code,derived_loan_product_type,derived_ethnicity,derived_race,derived_sex,purchaser_type,preapproval,loan_type,...,aus_3,aus_4,aus_5,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2023,549300RN01LBYR8ZVX74,42125,Conventional:First Lien,Not Hispanic or Latino,White,Female,0,2,1,...,NaN,NaN,NaN,5319,7.56,101900,127.88,1698,1929,43
1,2023,549300RN01LBYR8ZVX74,42027,Conventional:First Lien,Not Hispanic or Latino,White,Joint,71,2,1,...,NaN,NaN,NaN,2983,18.44,115500,119.02,967,1393,51
2,2023,549300RN01LBYR8ZVX74,42043,FHA:First Lien,Hispanic or Latino,Race Not Available,Female,71,2,2,...,NaN,NaN,NaN,6484,40.72,102700,106.86,1388,1929,42
3,2023,549300RN01LBYR8ZVX74,42087,Conventional:First Lien,Not Hispanic or Latino,White,Joint,0,2,1,...,NaN,NaN,NaN,2372,8.26,77800,53.55,613,959,73
4,2023,549300RN01LBYR8ZVX74,42061,Conventional:First Lien,Not Hispanic or Latino,White,Female,0,2,1,...,NaN,NaN,NaN,4624,5.95,77800,90.06,1508,2212,51
